### Modules for Bertrand competetion

In [57]:
from numba import njit
from typing import List
import numpy as np
from itertools import combinations
from numba import njit
from typing import List

In [11]:
# Demand funktion for n-spillere. 
@njit
def demand(prices: List[float]) -> float:
    num_prices = len(prices)
    num_lower = 0
    num_equal = 0
    
    for price in prices[1:]:
        if prices[0] > price:
            num_lower += 1
        elif prices[0] == price:
            num_equal += 1
    
    # Check om nogen priser er lavere end firma 0. 
    if num_lower == num_prices - 1 and num_equal == 0:
        return 0.0
    # Hvis ikke, check hcor mange firmaer er lig firma 0.
    elif num_equal > 0:
        return 1.0 / (num_equal+1) * (1-prices[0])
    # Ellers for firma 0 hele markedet. 
    else:
        return 1.0 - prices[0]

In [12]:
# Profit funktion for n-spillere. 
@njit
def profit(prices: List[float]) -> List[float]:
    return prices[0] * demand(prices)

In [58]:
# Test
# prices = [0.5, 0.5, 0.5]

# profit = profit(prices)
# demand = demand(prices)

# print("Demand:", demand)
# print("Profit:", profit)


Demand: 0.16666666666666666
Profit: 0.08333333333333333


C:\Users\sofus\AppData\Local\Temp\ipykernel_6880\2782928107.py:8: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'prices' of function 'demand'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_6880\2782928107.py", line 11:
<source missing, REPL/exec in use?>

  return prices[0] * demand(prices)
C:\Users\sofus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'prices' of function 'profit'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-s

In [14]:
# Definer set_price funktionen
# Ikke sikker på at dette virker. 
def set_price(i, t, p_table, Q_table, prices, epsilon):
    if epsilon >= np.random.uniform(0, 1):
        return np.random.choice(prices)
    else:
        num_players = len(prices)
        num_states = len(p_table)

        # Find the indices of the opponents' prices
        opponent_prices = [p_table[(i + j) % num_players, t-1] for j in range(1, num_players)]
        opponent_indices = [np.where(prices == price)[0][0] for price in opponent_prices]

        # Find the indices of the current state and other opponent's state
        state_indices = [np.where(prices == p_table[k, t-1])[0][0] for k in range(num_states) if k != i]

        maxedQ_idx = np.argmax(Q_table[state_indices[1], state_indices[0], :])
        return prices[maxedQ_idx]


In [15]:
# Definer curr_profit funktionen
# Ikke sikker på dette virker. 
@njit
def curr_prof(p_table, profits, i, t):
    num_players = p_table.shape[0]
    # more dimensions can be added
    j = (i + 1) % num_players
    k = (i + 2) % num_players
    # add to profit by adding p_table[x, t]
    profits[i, t] = profit(p_table[i, t], p_table[j, t], p_table[k, t])
    return


### Competetive benchmark

In [56]:
# Find unikke kombinationer. 
def find_unique_combinations(lst, n):
    unique_combinations = set()
    for combination in combinations(lst, n):
        unique_combinations.add(tuple(sorted(combination)))
    return unique_combinations

# Setup prisgrids for n-spillere. 
two_player = np.linspace(0, 4/6, 5) #[0, 1/6, 2/6, 3/6, 4/6]
three_player = np.linspace(0, 5/6, 6) #[0, 1/6, 2/6, 3/6, 4/6, 5/6]
unique_representations_two = find_unique_combinations(two_player, 2)
unique_representations_three = find_unique_combinations(three_player, 2)

# Beregn profitter og gem i en liste.  
unique_profits = set()
for combination in unique_representations_two:
    unique_profits.add(profit(combination))
# Beregn totale profit
total_profit = sum(unique_profits)
# Regn gennemsnitlig pris pr. periode pr. firma
avg_profit_two = total_profit/2/len(two_player)

# Beregn profitter og gem i en liste.  
unique_profits = set()
for combination in unique_representations_three:
    unique_profits.add(profit(combination))
# Beregn totale profit
total_profit = sum(unique_profits)
# Regn gennemsnitlig pris pr. periode pr. firma
avg_profit_three = total_profit/3/len(three_player)

#Print
print('Competetive benchmark 2 players:', avg_profit_two)
print('Competetive benchmark 3 players:', avg_profit_three)

Competetive benchmark 2 players: 0.061111111111111116
Competetive benchmark 3 players: 0.0462962962962963
